In [1]:
from utils import RL_with_constraint, RL_without_constraint, RLWKC_with_constraint, RLWKC_without_constraint, HTP, sparsePermutation, addNoise, meanNormalizedError, sgn, one_norm, only_priors
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from IPython.display import clear_output
import random  
import scipy
np.random.seed(90)
random.seed(90)

In [2]:
n_Permuted = 16                                           # number of permutation
cv_measurements = 5
m = 32                                                    # number of known correspondences
noise_fraction = 0.02
permutation_runs, noise_runs = 50, 50
p = 240                                                   # dimension of x
k = 14                                                    # sparsity of x
N_Metrics = np.zeros(shape = (permutation_runs, noise_runs, 9))
x = np.load("x_sparsity_" + str(k) + ".npy")
A_cv =  np.load("A_cv_fixed.npy")  
y_cv = addNoise(A_cv @ x, noise_fraction)     


for N in [80]:
    A = np.load("A_fixed.npy")[:N, :]           # extract N rows from the generated matrix

    for per_run in range(permutation_runs):
        print("N: {}, Permutation run: {}".format(N, per_run))
        P = sparsePermutation(N, m, n_Permuted)
        noiseLess_y = P @ A @ x
        z_true = noiseLess_y - A @ x
        max_abs_z = np.max(np.abs(z_true))
        sum_abs_z = np.sum(np.abs(z_true))

        l1_RLWKC_with_constraint, l2_RLWKC_with_constraint = None, None
        l1_RLWKC_without_constraint, l2_RLWKC_without_constraint = None, None
        l1_RL_with_constraint, l2_RL_with_constraint = None, None
        l1_RL_without_constraint, l2_RL_without_constraint = None, None
        l1_one_norm = None
        l1_only_priors = None

        for noise_run in range(noise_runs):
            print("N: {}, Noisy run: {}".format(k, noise_run))
            noisy_y = addNoise(noiseLess_y, noise_fraction)
            xhat_RLWKC_with_constraint, l1_RLWKC_with_constraint, l2_RLWKC_with_constraint = RLWKC_with_constraint(A, noisy_y, A_cv, y_cv, m, l1_RLWKC_with_constraint, l2_RLWKC_with_constraint)
            xhat_RLWKC_without_constraint, l1_RLWKC_without_constraint, l2_RLWKC_without_constraint = RLWKC_without_constraint(A, noisy_y, A_cv, y_cv, m, l1_RLWKC_without_constraint, l2_RLWKC_without_constraint)
            xhat_RL_with_constraint, l1_RL_with_constraint, l2_RL_with_constraint = RL_with_constraint(A, noisy_y, A_cv, y_cv, l1_RL_with_constraint, l2_RL_with_constraint)
            xhat_RL_without_constraint, l1_RL_without_constraint, l2_RL_without_constraint = RL_without_constraint(A, noisy_y, A_cv, y_cv, l1_RL_without_constraint, l2_RL_without_constraint)
            xhat_HTP = HTP(np.concatenate((A, A_cv), axis = 0), np.concatenate((noisy_y, y_cv), axis = 0), k)
            xhat_one_norm, l1_one_norm = one_norm(A, noisy_y, A_cv, y_cv, l1_one_norm)
            xhat_only_priors, l1_only_priors = only_priors(A, noisy_y, A_cv, y_cv, m, l1_only_priors)
  
            N_Metrics[per_run][noise_run][0] = meanNormalizedError(x, xhat_RLWKC_with_constraint)
            N_Metrics[per_run][noise_run][1] = meanNormalizedError(x, xhat_RLWKC_without_constraint)
            N_Metrics[per_run][noise_run][2] = meanNormalizedError(x, xhat_RL_with_constraint)
            N_Metrics[per_run][noise_run][3] = meanNormalizedError(x, xhat_RL_without_constraint)
            N_Metrics[per_run][noise_run][4] = meanNormalizedError(x, xhat_HTP)
            N_Metrics[per_run][noise_run][5] = meanNormalizedError(x, xhat_one_norm)
            N_Metrics[per_run][noise_run][6] = meanNormalizedError(x, xhat_only_priors)
            N_Metrics[per_run][noise_run][7] = max_abs_z
            N_Metrics[per_run][noise_run][8] = sum_abs_z
           

            print(N_Metrics[per_run][noise_run][0], N_Metrics[per_run][noise_run][1], N_Metrics[per_run][noise_run][2], N_Metrics[per_run][noise_run][3], N_Metrics[per_run][noise_run][4], N_Metrics[per_run][noise_run][5], N_Metrics[per_run][noise_run][6], N_Metrics[per_run][noise_run][7], N_Metrics[per_run][noise_run][8])

    np.save('N_Metrics' + str(N) + '.npy', N_Metrics)

N: 80, Permutation run: 0
N: 14, Noisy run: 0
Choose lambda1 properly
Choose lambda1 properly
0.0486359227617831 0.0451925147745336 0.2576285279512593 0.254101118756658 0.7073874394337163 0.5566867976866919 0.7227288639687652 9.442972307785281 44.192026046531616
N: 14, Noisy run: 1
Choose lambda1 properly
Choose lambda1 properly
0.08658493745338812 0.08693889468223583 0.26430317627219385 0.2710093426359463 0.7007407275883527 0.571918258156623 0.7192714755364308 9.442972307785281 44.192026046531616
N: 14, Noisy run: 2
Choose lambda1 properly
Choose lambda1 properly
0.06566478418957684 0.08033219195451632 0.26310785019742483 0.24361193791051516 0.640653700075511 0.5558136807110009 0.6549720880909771 9.442972307785281 44.192026046531616
N: 14, Noisy run: 3
Choose lambda1 properly
Choose lambda1 properly
0.07787108482249432 0.055201580649642236 0.26982688397015314 0.2822041598133883 0.644083316683387 0.5463264271477418 0.7863007498160556 9.442972307785281 44.192026046531616
N: 14, Noisy ru